In [1]:
!pip install yfinance timesfm==1.1.0


INFO: pip is looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warning

In [2]:
!pip install yfinance
!pip install utilsforecast


In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['JAX_PMAP_USE_TENSORSTORE'] = 'false'

In [2]:
import timesfm
import gc
import numpy as np
import pandas as pd
from timesfm import patched_decoder
from timesfm import data_loader
import yfinance as yf

2024-11-12 20:15:13.715875: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
import numpy as np
import tensorflow as tf
import random
import os

# Set a seed value
seed_value = 42

# Set Python random seed
random.seed(seed_value)

# Set NumPy random seed
np.random.seed(seed_value)

# Set TensorFlow random seed
tf.random.set_seed(seed_value)

# For reproducibility in other areas, like OS operations
os.environ['PYTHONHASHSEED'] = str(seed_value)


In [4]:
from tqdm import tqdm
import dataclasses
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [5]:
# Loading TimesFM pretrained checkpoint
tfm = timesfm.TimesFm(
    context_len=64,
    horizon_len=1,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")



2024-11-12 20:15:41.095038: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.6.77). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Constructing model weights.


Constructed model weights in 5.01 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 4.54 seconds.
Jitting decoding.
Jitted decoding in 22.56 seconds.


In [6]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# **Step 1: Load and Preprocess Data for All Stocks**

# List of stock tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'V', 'NFLX']

# Load daily stock data for each ticker
all_data = []

for idx, ticker in enumerate(tickers, start=1):
    data = yf.download(ticker, start='2012-05-18', end='2024-01-01', interval='1d')
    if data.empty:
        print(f"Failed to download data for {ticker}")
        continue
    data.index = pd.to_datetime(data.index)
    data = data.dropna()
    data.reset_index(inplace=True)
    # Handle MultiIndex columns if present
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(1)
    data.rename(columns={'Date': 'date', 'Close': 'y'}, inplace=True)
    data['ticker'] = idx  # Replace ticker name with a unique number

    final_cols = ["date", "y", "ticker"]
    data = data[final_cols]
    all_data.append(data)

# Combine all stock data into one DataFrame
combined_data = pd.concat(all_data)

print(f"Total combined data length: {len(combined_data)}")

# **Step 2: Separate Apple Data (ticker == 1)**

apple_data = combined_data[combined_data['ticker'] == 1].copy()

# **Step 3: Define Validation and Test Periods for Apple**

# Define date ranges
val_start_date = '2022-07-01'
val_end_date = '2022-12-31'
test_start_date = '2023-01-01'

# Ensure 'date' column is in datetime format
apple_data['date'] = pd.to_datetime(apple_data['date'])

# Create validation data for Apple
val_data = apple_data[(apple_data['date'] >= val_start_date) & (apple_data['date'] <= val_end_date)]

# Create test data for Apple
test_data = apple_data[apple_data['date'] >= test_start_date]

# Create training data for Apple (excluding validation and test periods)
train_data_apple = apple_data[apple_data['date'] < val_start_date]

# **Step 4: Create Training Data for Non-Apple Stocks**

non_apple_data = combined_data[combined_data['ticker'] != 1].copy()

# **Step 5: Combine Training Data**

train_data = pd.concat([train_data_apple, non_apple_data])

print(f"Training data length: {len(train_data)}")
print(f"Validation data length: {len(val_data)}")
print(f"Test data length: {len(test_data)}")

# **Step 6: Scale Only the Training Data**

# Create a dictionary to store scalers for each ticker
scalers = {}

def scale_data(data, scalers, fit=False):
    data=data.reset_index(drop=True)
    data_scaled = data.copy().reset_index(drop=True)
    for ticker in data['ticker'].unique():
        ticker_data = data[data['ticker'] == ticker]
        if fit:
            scaler = StandardScaler()
            scaler.fit(ticker_data[['y']])
            scalers[ticker] = scaler
        else:
            scaler = scalers.get(ticker)

        data_scaled.loc[ticker_data.index, 'y_scaled'] = scaler.transform(ticker_data[['y']])

    return data_scaled

# Fit scalers on training data
train_data_scaled = scale_data(train_data, scalers, fit=True)

# Scale validation and test data using the fitted scalers
val_data_scaled = scale_data(val_data, scalers, fit=False)
test_data_scaled = scale_data(test_data, scalers, fit=False)

# **Step 7: Combine All Data and Reset Index**

full_data = pd.concat([train_data_scaled, val_data_scaled, test_data_scaled]).reset_index(drop=True)
print(f"Total data length after combining: {len(full_data)}")

# **Step 8: Compute Index Ranges for the Data Loader**

# Calculate cumulative lengths
train_end_idx = len(train_data_scaled)
val_end_idx = train_end_idx + len(val_data_scaled)
total_length = len(full_data)

print(f"Train end index: {train_end_idx}")
print(f"Validation end index: {val_end_idx}")
print(f"Total data length: {total_length}")

# **Define Context Length and Prediction Length**

context_len = 64  # Historical length
pred_len = 1      # Prediction length
freq = 'D'        # Daily frequency

# **Compute Ranges**

train_range = [0, train_end_idx + context_len]
val_range = [train_end_idx + context_len, val_end_idx + context_len]
test_range = [val_end_idx + context_len, total_length]

print(f"Train range: {train_range}")
print(f"Validation range: {val_range}")
print(f"Test range: {test_range}")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Total combined data length: 29230
Training data length: 28853
Validation data length: 127
Test data length: 250
Total data length after combining: 29230
Train end index: 28853
Validation end index: 28980
Total data length: 29230
Train range: [0, 28917]
Validation range: [28917, 29044]
Test range: [29044, 29230]


In [7]:
full_data

Price,date,y,ticker,y_scaled
0,2012-05-18 00:00:00+00:00,18.942142,1,-0.816202
1,2012-05-21 00:00:00+00:00,20.045713,1,-0.791602
2,2012-05-22 00:00:00+00:00,19.891787,1,-0.795033
3,2012-05-23 00:00:00+00:00,20.377144,1,-0.784214
4,2012-05-24 00:00:00+00:00,20.190001,1,-0.788386
...,...,...,...,...
29225,2023-12-22 00:00:00+00:00,193.600006,1,3.077133
29226,2023-12-26 00:00:00+00:00,193.050003,1,3.064873
29227,2023-12-27 00:00:00+00:00,193.149994,1,3.067102
29228,2023-12-28 00:00:00+00:00,193.580002,1,3.076687


In [8]:
data_path = '/content/stocks_data.csv'
full_data.to_csv(data_path, index=False)

In [9]:
freq = 'D'
context_len = 64
pred_len = 1
batch_size = 32



In [10]:


num_ts = 1
# Create TimeSeriesdata loader
dtl = data_loader.TimeSeriesdata(
      data_path=data_path,
      datetime_col="date",
      num_cov_cols=None,
      cat_cov_cols=None,
      ts_cols=["y_scaled"],
      train_range=train_range,
      val_range=val_range,
      test_range=test_range,
      hist_len=context_len,
      pred_len=pred_len,
      batch_size=1,  # Change as per requirement
      freq=freq,
      normalize=False,
      epoch_len=None,
      holiday=False,
      permute=False,
  )
train_batches = dtl.tf_dataset(mode="train", shift=1).batch(batch_size)
val_batches = dtl.tf_dataset(mode="val", shift=pred_len)
test_batches = dtl.tf_dataset(mode="test", shift=pred_len)

2024-11-12 20:16:35.653572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-11-12 20:16:35.653853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-11-12 20:16:35.654553: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-11-12 20:16:35.654730: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such 

In [11]:
for tbatch in tqdm(train_batches.as_numpy_iterator()):
    pass
print(tbatch[0].shape)
print(tbatch[1].shape)
print(tbatch[2].shape)

1804it [00:26, 68.47it/s] 

(8, 1, 64)
(8, 8, 64)
(8, 1, 64)


# MAE on the test split for the pretrained TimesFM model

In [12]:
mse_losses = []
for batch in tqdm(test_batches.as_numpy_iterator()):
    past = batch[0]
    actuals = batch[3]

    _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
    forecasts = forecasts[:, 0 : actuals.shape[1], 5]

    mse_losses.append(np.square(forecasts - actuals).mean())

print(f"MSE: {np.mean(mse_losses)}")


186it [00:03, 52.22it/s]

MSE: 0.0026948237791657448


# Fine-tuning the model

In [13]:
import jax
from jax import numpy as jnp
from praxis import pax_fiddle
from praxis import py_utils
from praxis import pytypes
from praxis import base_model
from praxis import optimizers
from praxis import schedules
from praxis import base_hyperparams
from praxis import base_layer
from paxml import tasks_lib
from paxml import trainer_lib
from paxml import checkpoints
from paxml import learners
from paxml import partitioning
from paxml import checkpoint_types

In [14]:
NestedMap = py_utils.NestedMap
WeightInit = base_layer.WeightInit
WeightHParams = base_layer.WeightHParams
InstantiableParams = py_utils.InstantiableParams
JTensor = pytypes.JTensor
NpTensor = pytypes.NpTensor
WeightedScalars = pytypes.WeightedScalars
instantiate = base_hyperparams.instantiate
LayerTpl = pax_fiddle.Config[base_layer.BaseLayer]
AuxLossStruct = base_layer.AuxLossStruct

AUX_LOSS = base_layer.AUX_LOSS
template_field = base_layer.template_field

# Standard prng key names
PARAMS = base_layer.PARAMS
RANDOM = base_layer.RANDOM

key = jax.random.PRNGKey(seed=1234)

In [15]:
model = pax_fiddle.Config(
    patched_decoder.PatchedDecoderFinetuneModel,
    name='patched_decoder_finetune',
    core_layer_tpl=tfm.model_p,
)

In [16]:

@pax_fiddle.auto_config
def build_learner() -> learners.Learner:
  return pax_fiddle.Config(
      learners.Learner,
      name='learner',
      loss_name='avg_qloss',
      optimizer=optimizers.Adam(
          epsilon=1e-7,
          clip_threshold=1e2,
          learning_rate=1e-2,
          lr_schedule=pax_fiddle.Config(
              schedules.Cosine,
              initial_value=1e-3,
              final_value=1e-4,
              total_steps=40000,
          ),
          ema_decay=0.9999,
      ),
      # Linear probing i.e we hold the transformer layers fixed.
      bprop_variable_exclusion=['.*/stacked_transformer_layer/.*'],
  )

In [17]:

task_p = tasks_lib.SingleTask(
    name='ts-learn',
    model=model,
    train=tasks_lib.SingleTask.Train(
        learner=build_learner(),
    ),
)

In [18]:
task_p.model.ici_mesh_shape = [1, 1, 1]
task_p.model.mesh_axis_names = ['replica', 'data', 'mdl']

DEVICES = np.array(jax.devices()).reshape([1, 1, 1])
MESH = jax.sharding.Mesh(DEVICES, ['replica', 'data', 'mdl'])

num_devices = jax.local_device_count()
print(f'num_devices: {num_devices}')
print(f'device kind: {jax.local_devices()[0].device_kind}')

num_devices: 1
device kind: Tesla T4


In [19]:
jax_task = task_p
key, init_key = jax.random.split(key)

# To correctly prepare a batch of data for model initialization (now that shape
# inference is merged), we take one devices*batch_size tensor tuple of data,
# slice out just one batch, then run the prepare_input_batch function over it.


def process_train_batch(batch):

    past_ts = batch[0].reshape(len(batch[2]) * num_ts, -1)
    actual_ts = batch[3].reshape(len(batch[2]) * num_ts, -1)
    return NestedMap(input_ts=past_ts, actual_ts=actual_ts)


def process_eval_batch(batch):
    past_ts = batch[0]
    actual_ts = batch[3]
    return NestedMap(input_ts=past_ts, actual_ts=actual_ts)


jax_model_states, _ = trainer_lib.initialize_model_state(
    jax_task,
    init_key,
    process_train_batch(tbatch),
    checkpoint_type=checkpoint_types.CheckpointType.GDA,
)

In [20]:
jax_model_states.mdl_vars['params']['core_layer'] = tfm._train_state.mdl_vars['params']
jax_vars = jax_model_states.mdl_vars
gc.collect()

0

In [21]:
jax_task = task_p


def train_step(states, prng_key, inputs):
  return trainer_lib.train_step_single_learner(
      jax_task, states, prng_key, inputs
  )


def eval_step(states, prng_key, inputs):
  states = states.to_eval_state()
  return trainer_lib.eval_step_single_learner(
      jax_task, states, prng_key, inputs
  )

key, train_key, eval_key = jax.random.split(key, 3)
train_prng_seed = jax.random.split(train_key, num=jax.local_device_count())
eval_prng_seed = jax.random.split(eval_key, num=jax.local_device_count())

p_train_step = jax.pmap(train_step, axis_name='batch')
p_eval_step = jax.pmap(eval_step, axis_name='batch')

In [22]:
replicated_jax_states = trainer_lib.replicate_model_state(jax_model_states)
replicated_jax_vars = replicated_jax_states.mdl_vars

In [23]:
best_eval_loss = 1e7
step_count = 0
patience = 0
NUM_EPOCHS = 100
PATIENCE = 5
TRAIN_STEPS_PER_EVAL = 1000
CHECKPOINT_DIR='/content'

In [28]:
def reshape_batch_for_pmap(batch, num_devices):
  def _reshape(input_tensor):
    bsize = input_tensor.shape[0]
    residual_shape = list(input_tensor.shape[1:])
    nbsize = bsize // num_devices
    return jnp.reshape(input_tensor, [num_devices, nbsize] + residual_shape)

  return jax.tree.map(_reshape, batch)

In [24]:
!rm -rf /content/checkpoint*


In [29]:
import pickle
save_count = 0
for epoch in range(NUM_EPOCHS):
    print(f"__________________Epoch: {epoch}__________________", flush=True)

    # Train phase
    train_its = train_batches.as_numpy_iterator()
    if patience >= PATIENCE:
        print("Early stopping.", flush=True)
        break

    train_losses = []  # Initialize train losses for the epoch
    for batch in train_its:
        if patience >= PATIENCE:
            print("Early stopping.", flush=True)
            break

        tbatch = process_train_batch(batch)
        tbatch = reshape_batch_for_pmap(tbatch, num_devices)
        replicated_jax_states, step_fun_out = p_train_step(
            replicated_jax_states, train_prng_seed, tbatch
        )
        train_losses.append(step_fun_out.loss[0])

        # Increment step count
        step_count += 1

    # Calculate and print average train loss for the epoch
    avg_train_loss = np.mean(train_losses)
    print(f"Train loss at end of epoch {epoch}: {avg_train_loss}", flush=True)

    # Eval phase at the end of the epoch
    print("Starting eval.", flush=True)
    val_its = val_batches.as_numpy_iterator()
    eval_losses = []  # Initialize eval losses for the epoch

    for ev_batch in tqdm(val_its):
        ebatch = process_eval_batch(ev_batch)
        ebatch = reshape_batch_for_pmap(ebatch, num_devices)
        _, step_fun_out = p_eval_step(replicated_jax_states, eval_prng_seed, ebatch)
        eval_losses.append(step_fun_out.loss[0])

    # Calculate and print average eval loss for the epoch
    mean_loss = np.mean(eval_losses)
    print(f"Eval loss at end of epoch {epoch}: {mean_loss}", flush=True)

    # Check for checkpoint saving and patience logic
    if mean_loss < best_eval_loss or np.isnan(mean_loss):
        best_eval_loss = mean_loss
        print("Saving checkpoint.")
        jax_state_for_saving = py_utils.maybe_unreplicate_for_fully_replicated(
            replicated_jax_states
        )
        save_count += 1
        if save_count % 5 == 0:
            print("Unwaited save to Google Drive.")
            checkpoints.save_checkpoint(
                jax_state_for_saving, '/content/drive/MyDrive', overwrite=True,
            )
            with open(os.path.join( '/content/drive/MyDrive', 'train_state_info.pkl'), 'wb') as f:
               pickle.dump({'step_count': step_count, 'epoch': epoch}, f)
               print(f"Checkpoint saved at step {step_count}, epoch {epoch}")
        checkpoints.save_checkpoint(
            jax_state_for_saving, CHECKPOINT_DIR, overwrite=True
        )
        patience = 0  # Reset patience since we improved
        del jax_state_for_saving
        gc.collect()
    else:
        patience += 1
        print(f"patience: {patience}")

    # Early stopping check
    if patience >= PATIENCE:
        print("Early stopping.", flush=True)
        break


__________________Epoch: 0__________________
Train loss at end of epoch 0: 0.19216634333133698
Starting eval.


127it [00:13,  9.13it/s]

Eval loss at end of epoch 0: 0.39031529426574707



        shapes of train_state  as the unpadded shapes.


Saving checkpoint.


...

Train loss at end of epoch 16: 0.15191251039505005
Starting eval.


127it [00:01, 100.65it/s]


Eval loss at end of epoch 16: 0.3791418969631195
patience: 4
__________________Epoch: 17__________________
Train loss at end of epoch 17: 0.15103602409362793
Starting eval.


127it [00:01, 102.15it/s]

Eval loss at end of epoch 17: 0.37777671217918396
patience: 5
Early stopping.


In [30]:
train_state = checkpoints.restore_checkpoint(jax_model_states, CHECKPOINT_DIR)
print(train_state.step)
tfm._train_state.mdl_vars['params'] = train_state.mdl_vars['params']['core_layer']
tfm.jit_decode()

ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


23452
Jitting decoding.
Jitted decoding in 20.48 seconds.


In [31]:
mse_losses = []
for batch in tqdm(test_batches.as_numpy_iterator()):
    past = batch[0]
    actuals = batch[3]
    _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
    forecasts = forecasts[:, 0 : actuals.shape[1], 5]
    mse_losses.append(np.square(forecasts - actuals))

print(f"Mse: {np.mean(mse_losses)}")

186it [00:03, 57.69it/s]

Mse: 0.0023281839676201344


In [32]:
from collections import defaultdict
# Download stock data from Yahoo Finance
ticker = 'AAPL'  # Example: Apple Inc.
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")
data = data.dropna()

df = data[['Close']].reset_index()
df.columns = ['date', 'close']
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


def get_batched_data_fn(batch_size: int = 32, context_len: int = 32, horizon_len: int = 1):
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(df) - (context_len + horizon_len), horizon_len):
        num_examples += 1
        context_end = start + context_len
        examples["inputs"].append(df["close"][start:context_end].tolist())
        examples["outputs"].append(df["close"][context_end:context_end + horizon_len].tolist())
        examples["dates"].append(df.index[start:context_end + horizon_len].tolist())  # Add dates to examples

    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size): ((i + 1) * batch_size)] for k, v in examples.items()}

    return data_fn
context_len=64
input_patch_len=32
horizon_len=1
batch_size=1

input_data = get_batched_data_fn(batch_size=batch_size, context_len=context_len, horizon_len=horizon_len)

results = []
# Process in smaller sub-batches
for i, example in enumerate(input_data()):
    raw_forecast, _ = tfm.forecast(
        inputs=example["inputs"], freq=[0] * len(example["inputs"])
    )
    for j in range(len(example["inputs"])):
        result = {
            'input_index': j,
            'train_start_date': str(example["dates"][j][0]),
            'train_end_date': str(example["dates"][j][-2]),
            'predict_date': str(example["dates"][j][-1]),
            'raw_forecast': raw_forecast[j][0],
            'y_actual': example["outputs"][j][0],
            'batch_size': batch_size,
            'context_len': context_len,
            'input_patch_len': input_patch_len
        }
        results.append(result)
results_df = pd.DataFrame(results)
from sklearn.metrics import mean_squared_error, accuracy_score
mean_squared_error(results_df['y_actual'], results_df['raw_forecast'])


[*********************100%***********************]  1 of 1 completed


4.742681289884589